Clean Auction games
--------------------

In [3]:
import glob
import pandas as pd

from tqdm import tqdm

In [4]:
import numpy as np

In [5]:
dfs = []
for filename in glob.glob('/home/nightwing/rsc/Literature-Article/src/data/auction/*.json'):
    dfs.append(pd.read_json(filename))

In [6]:
auction_df = pd.concat(dfs, ignore_index=True)
auction_df.tail(5)

,abstract,author,date,journal,key,key_word,pages,provenance,score,title,unique_key
15999,We study the problem of characterizing reven...,Vineet Abhishek,2010,arXiv,Abhishek2010,NaN,NaN,arXiv,NaN,Revenue Optimal Auction for Single-Minded Buyers,d95c404a35ae66006fbb3ae89c5dbe71
16000,We study the problem of characterizing reven...,Bruce Hajek,2010,arXiv,Abhishek2010,NaN,NaN,arXiv,NaN,Revenue Optimal Auction for Single-Minded Buyers,d95c404a35ae66006fbb3ae89c5dbe71
16001,We study efficiency loss in Bayesian revenue...,Vineet Abhishek,2010,arXiv,Abhishek2010,NaN,NaN,arXiv,NaN,Efficiency Loss in Revenue Optimal Auctions,6bc0926da61cb7246c9964d32aeeb892
16002,We study efficiency loss in Bayesian revenue...,Bruce Hajek,2010,arXiv,Abhishek2010,NaN,NaN,arXiv,NaN,Efficiency Loss in Revenue Optimal Auctions,6bc0926da61cb7246c9964d32aeeb892
16003,The Generalized Second Price auction is the ...,Brendan Lucier,2010,arXiv,Lucier2010,NaN,NaN,arXiv,NaN,Improved Social Welfare Bounds for GSP at Equi...,9d5226b9c0cb03597ca7881d44193871


In [7]:
auction_df = auction_df[~auction_df['title'].isnull()]

In [8]:
size = len(auction_df['title'].unique())
size

2227

Cleaning the data set.
---------------------

**1. Removing articles.**
Check whether the key `auction` is within title or absract. If not remove from data set.

In [13]:
key = 'auction'

In [14]:
auction_df['title'] = auction_df['title'].str.lower()

In [15]:
not_in_title = set()
for i, row in auction_df.iterrows():
    if key not in row['title']:
        not_in_title.add(i)

In [16]:
not_in_abstract = set()
for i, row in auction_df.iterrows():
    if row['abstract']:
        if key not in row['abstract']:
            not_in_abstract.add(i)

In [17]:
drop = list(not_in_abstract.intersection(not_in_title))

In [18]:
auction_df = auction_df[~auction_df.index.isin(drop)]

In [22]:
size = len(auction_df['title'].unique())
size

2103

In [26]:
auction_df = auction_df.replace(2018, 2017)

**Authors.** Cleaning authors.

The Levenshtein distance is used as a mesaure of similarity. A lot of manual work was put into this.

In [23]:
from fuzzywuzzy import fuzz

In [24]:
auction_df['author'] = auction_df['author'].str.lower()

In [25]:
len(auction_df['author'].unique())

4168

In [26]:
auction_df.describe()

,date,score
count,15811.000000,70.000000
mean,2010.080197,13.769912
std,5.066317,3.835304
min,1974.000000,9.449580
25%,2007.000000,9.706198
50%,2011.000000,13.591616
75%,2014.000000,14.874890
max,2018.000000,21.685135


In [29]:
pairs = []
for count, i in tqdm(enumerate(auction_df.author.unique())):
    for j in auction_df.author.unique():
        ratio = fuzz.token_set_ratio(i,j)
        if ratio >=85 and ratio != 100:
            pairs.append((i, j))

4168it [08:53,  7.81it/s]


In [30]:
len(pairs)

984

In [31]:
df = pd.DataFrame(pairs, columns=['first_suggestion', 'second_suggestion'])
df.head(10)

,first_suggestion,second_suggestion
0,zheng tao,tao zhang
1,jakobsen thomas,r. thomas
2,eva tardos,éva tardos
3,ning chen,jing chen
4,ning chen,chen jing
5,moshe babaioff,m. babaioff
6,liad blumrosen,l. blumrosen
7,zhong chen,hongli chen
8,nicole immorlica,nicole immolica
9,wang qing,ping wang


In [32]:
df.to_csv('/home/nightwing/rsc/Literature-Article/src/data/auction/pairs.csv')

In [33]:
replacements_df = pd.read_csv('/home/nightwing/rsc/Literature-Article/src/data/auction/replacements.csv')

In [34]:
replacements_df.head()

,Unnamed: 0,first_suggestion,second_suggestion,check
0,0,zheng tao,tao zhang,False
1,1,jakobsen thomas,r. thomas,False
2,2,eva tardos,éva tardos,True
3,3,ning chen,jing chen,False
4,4,ning chen,chen jing,False


In [37]:
for i, replacement_row in replacements_df.iterrows():
    if replacement_row['check']:
        auction_df = auction_df.replace(replacement_row['second_suggestion'], 
                                        replacement_row['first_suggestion'])

In [38]:
auction_df.to_csv('/home/nightwing/rsc/Literature-Article/src/data/auction_df.csv')